In [2]:
import pandas as pd

In [14]:
def openarff(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # Find where the actual data starts
    data_start = next(i for i, line in enumerate(lines) if line.strip() == "@data") + 1

    # Read only the data part
    df_train = pd.read_csv(
        filepath,
        skiprows=data_start, 
        header=None
    )

    return df_train

In [ ]:
df_test = openarff(r'C:..\5_Data\Epilepsy\Epilepsy_TEST.ts')

In [16]:
df_test.iloc[:, -1] = df_test.iloc[:, -1].str.split(":").str[1]  # Extract label
label_counts = df_test.iloc[:, -1].value_counts()  
print(label_counts)


615
WALKING     37
RUNNING     37
EPILEPSY    34
SAWING      30
Name: count, dtype: int64


In [25]:
df_train = openarff(r'..\5_Data\Epilepsy\Epilepsy_Train.ts')


In [20]:
df_train.iloc[:, -1] = df_train.iloc[:, -1].str.split(":").str[1]  # Extract label
label_counts = df_train.iloc[:, -1].value_counts()  
print(label_counts)


615
WALKING     37
RUNNING     36
EPILEPSY    34
SAWING      30
Name: count, dtype: int64


In [10]:
df_train.to_csv('Epilepsy_TRAIN.csv')

In [19]:
!pip install cbor2


You should consider upgrading via the 'C:\Users\TimPr\allcode\TinyML\TinyMLSeizureDetectionDevice\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
import cbor2

with open(r"C:\Users\TimPr\Downloads\noise.5k6tpqrc.cbor", "rb") as f:
    data = cbor2.load(f)

print(data)  # Output the parsed CBOR data


{'protected': {'ver': 'v1', 'alg': 'HS256'}, 'signature': '0000000000000000000000000000000000000000000000000000000000000000', 'payload': {'device_name': '2A:70:3B:06:E9:7D', 'device_type': 'ARDUINO_NANO33BLE', 'interval_ms': 10.0, 'sensors': [{'name': 'accX', 'units': 'm/s2'}, {'name': 'accY', 'units': 'm/s2'}, {'name': 'accZ', 'units': 'm/s2'}, {'name': 'gyrX', 'units': 'deg/s'}, {'name': 'gyrY', 'units': 'deg/s'}, {'name': 'gyrZ', 'units': 'deg/s'}, {'name': 'magX', 'units': 'uT'}, {'name': 'magY', 'units': 'uT'}, {'name': 'magZ', 'units': 'uT   '}], 'values': [[2.9287071228027344, -6.147711277008057, 6.913856029510498, -8.1787109375, 38.39111328125, 7.080078125, 45.83740234375, 5.11474609375, -60.33935546875], [3.046621561050415, -5.765836238861084, 6.593631267547607, 0.18310546875, 32.77587890625, 18.5546875, 45.83740234375, 5.11474609375, -60.33935546875], [2.9730000495910645, -5.231330871582031, 6.276399612426758, 12.14599609375, 8.1787109375, 36.865234375, 45.83740234375, 5.1147

In [21]:
data.keys()

dict_keys(['protected', 'signature', 'payload'])

In [34]:
data["payload"]

{'device_name': '2A:70:3B:06:E9:7D',
 'device_type': 'ARDUINO_NANO33BLE',
 'interval_ms': 10.0,
 'sensors': [{'name': 'accX', 'units': 'm/s2'},
  {'name': 'accY', 'units': 'm/s2'},
  {'name': 'accZ', 'units': 'm/s2'},
  {'name': 'gyrX', 'units': 'deg/s'},
  {'name': 'gyrY', 'units': 'deg/s'},
  {'name': 'gyrZ', 'units': 'deg/s'},
  {'name': 'magX', 'units': 'uT'},
  {'name': 'magY', 'units': 'uT'},
  {'name': 'magZ', 'units': 'uT   '}],
 'values': [[2.9287071228027344,
   -6.147711277008057,
   6.913856029510498,
   -8.1787109375,
   38.39111328125,
   7.080078125,
   45.83740234375,
   5.11474609375,
   -60.33935546875],
  [3.046621561050415,
   -5.765836238861084,
   6.593631267547607,
   0.18310546875,
   32.77587890625,
   18.5546875,
   45.83740234375,
   5.11474609375,
   -60.33935546875],
  [2.9730000495910645,
   -5.231330871582031,
   6.276399612426758,
   12.14599609375,
   8.1787109375,
   36.865234375,
   45.83740234375,
   5.11474609375,
   -60.33935546875],
  [2.916736125

In [23]:
epi_dim1 = openarff('../5_Data/Epilepsy/EpilepsyDimension1_TEST.arff')
epi_dim2 = openarff('../5_Data/Epilepsy/EpilepsyDimension2_TEST.arff')
epi_dim3 = openarff('../5_Data/Epilepsy/EpilepsyDimension3_TEST.arff')


In [34]:
epi_dim1['dim'] = 'x'
epi_dim2['dim'] = 'y'
epi_dim3['dim'] = 'z'


In [35]:
# Check the shape of each dataset
print(epi_dim1.shape, epi_dim2.shape, epi_dim3.shape)

(138, 208) (138, 208) (138, 208)


In [ ]:
combined_epi = pd.concat([epi_dim1, epi_dim2, epi_dim3], axis=0)

In [45]:
combined_epi.head()

,0,1,2,3,4,5,6,7,8,9,...,198,199,200,201,202,203,204,205,206,dim
0,0.60,0.60,0.60,0.61,0.59,0.59,0.59,0.60,0.60,0.60,...,-0.83,-0.50,-0.49,-0.50,-0.53,-0.36,-0.51,-0.57,EPILEPSY,z
1,-0.35,-0.44,-0.86,-0.64,-0.45,-0.25,-0.08,-0.07,0.30,0.73,...,-0.10,-0.25,0.94,0.57,-0.07,-0.22,0.55,0.83,EPILEPSY,z
2,-0.08,-0.01,-0.53,-0.49,0.04,-0.44,-0.42,-0.04,0.04,-0.43,...,-0.79,-0.76,-0.77,-0.64,-0.65,-0.54,-0.62,-0.56,EPILEPSY,z
3,-0.43,-0.44,-0.47,-0.49,-0.53,-0.53,-0.57,-0.56,-0.51,-0.48,...,-0.43,-0.40,-0.71,-0.40,-0.75,-0.60,-0.46,-0.59,EPILEPSY,z
4,-0.30,-0.81,-0.55,-0.81,-0.13,-1.10,-0.59,-0.64,-0.25,-1.06,...,-0.28,-0.18,-0.14,-0.04,0.00,-0.02,-0.03,-0.02,EPILEPSY,z


In [41]:
combined_epi.to_csv('../5_Data/Epilepsy_TEST.csv')